# [Day 16](https://adventofcode.com/2020/day/16): Ticket Translation

In [1]:
import numpy as np
import re

with open("../data/16.txt", "r") as f:
    lines = [l.strip() for l in f.readlines()]

ranges = re.findall(r"(\d+)-(\d+)", "".join(lines[:20]))
lwr, upr = np.array(ranges, dtype=int).T.reshape(2, 20, 2)
myticket = np.array(lines[22].split(","), dtype=int)
tickets = np.array([t.split(",") for t in lines[25:]], dtype=int)

## Part 1

In [2]:
nums = tickets.reshape(tickets.size, 1, 1)
invalid = ((nums < lwr) | (upr < nums)).all(axis=(1, 2))
assert 25788 == np.sum(nums[invalid])

## Part 2

In [3]:
def enumfields(tickets, lwr, upr):
    infield = choice(maybefield(tickets, lwr, upr))
    return infield.argsort()

def maybefield(tickets, lwr, upr):
    t = tickets.T[..., np.newaxis, np.newaxis]
    return ((lwr <= t) & (t <= upr)).any(axis=-1).all(axis=1)

# cf. Day 21
def choice(x):
    ch, i = np.empty((len(x), 2), dtype=int), 0
    idx = np.einsum("ijk->jki", np.indices(x.shape))
    while len(x):
        j, k, sing = singular(x)
        ch[i:(i := i + k.sum())] = idx[sing]
        x, idx = x[(_jk := np.ix_(~j, ~k))], idx[_jk]
    return ch[:, 1][ch[:, 0].argsort()]

def singular(x):
    j = x.sum(axis=1) == 1
    k = x[j].sum(axis=0) == 1
    sing = np.zeros(x.shape, dtype=bool)
    sing[jk] = x[(jk := np.ix_(j, k))]
    return j, k, sing

valid = ~invalid.reshape(tickets.shape).any(axis=1)
departures = enumfields(tickets[valid], lwr, upr)[:6]
assert 3902565915559 == np.prod(myticket[departures])